# Supplementary information

This notebook provides supplementary information for the manuscript: "???"


# imports and custom functions

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')
from scipy.optimize import curve_fit
from scipy.stats import binned_statistic_2d
from tqdm import tqdm
import yaml
import xarray as xr
import hvplot.xarray
import urllib.request
import panel.widgets as pnw

In [ ]:
from holoviews import opts
opts.defaults(
    opts.Scatter(width=1200, height=400, tools=["hover"], show_grid=True, size=8),
    opts.Histogram(width=1000, height=400, tools=["hover"], ),
    opts.Image(frame_width=400, frame_height=400, tools=["hover"], xlabel="ToT1 (ns)", ylabel="ToT2 (ns)", fontscale=1.5),
    opts.Curve(width=800, height=500, tools=["hover"], show_grid=True, fontscale=1.5),
    opts.Points(width=1000, height=500, tools=["hover"], show_grid=True)
)

## custom functions

In [ ]:
offset = -28
def fit_single_scan(scan_nr: str) -> list:
    global df_scans
    model = lambda t, A, B, c: A * (1 - B * np.exp(-t / c))
    loss = lambda t, A, B, c: model(t, A, B, c)[0] / A

    # define our x-axis from the first scans
    x_axe = np.float_(runNrs["scan0100"]["delays"]) - 1

    # fetch data
    df = df_scans.query(f"`scan name` == '{scan_nr}'")

    # omit nans
    y = df[~df['ToT2 (ns)'].isna()]['ToT2 (ns)']
    if len(y) > -offset:
        y = y[offset:]
    x = x_axe[-len(y):]
    # unequal starting for fit
    #y = y[4:]
    #x = x_axe[-(len(y)):]
    #assert len(y) == len(x)

    p0 = [y[-6:].mean(), 1, 8]
    popt, _ = curve_fit(
        model,
        x, y,
        p0=p0,
        maxfev=5000,
        bounds=((50, 1e-2, 1), (2150, 1, 15)),
    )

    return [
        popt[2],  # recovery time
        df['ToT1 (ns)'][-6:].mean(),  # mean ToT1
        y[-6:].mean(),  # mean ToT2
        (popt[0] - model(np.array([2]), *popt))[0],  # absolute loss 2 µs
        (popt[0] - model(np.array([3]), *popt))[0],  # absolute loss 3 µs
        (popt[0] - model(np.array([4]), *popt))[0],  # absolute loss 4 µs
        (1 - loss(np.array([2]), *popt)) * 100,  # relative loss 2 µs
        (1 - loss(np.array([3]), *popt)) * 100,  # relative loss 3 µs
        (1 - loss(np.array([4]), *popt)) * 100,  # relative loss 4 µs
        (1 - loss(np.array([5]), *popt)) * 100,
        (1 - loss(np.array([6]), *popt)) * 100,
        (1 - loss(np.array([7]), *popt)) * 100,
        (1 - loss(np.array([8]), *popt)) * 100,
        (1 - loss(np.array([9]), *popt)) * 100,
        (1 - loss(np.array([10]), *popt)) * 100,
    ]

In [ ]:
get_x_axis_from_bins = lambda x_bins: 0.5 * (x_bins[1:] + x_bins[:-1])

# load data

In [ ]:
with urllib.request.urlopen("https://syncandshare.desy.de/index.php/s/zgPcWZMyBGcQ5MC/download/TOT_scans.yaml") as f:
    runNrs = yaml.safe_load(f)

In [ ]:
tot1s = {i['tot1'] for i in runNrs.values()}
tot2s = {i['tot2'] for i in runNrs.values()}

#scan_data = pd.read_parquet('https://syncandshare.desy.de/index.php/s/KRA42M7YimM3PAP/download/scan_data.parq')
scan_data = pd.read_parquet('data/scan_data.parq')
scan_data

In [ ]:
df_scans = pd.read_parquet('https://syncandshare.desy.de/index.php/s/Dw3Nf2PQ59QppYH/download/df_scans.parq')
#df_scans = pd.read_parquet('data/df_scans.parq')

## convert single traces to matrix

In [ ]:
tot1_tot2 = [(*fit_single_scan(scan), scan) for scan in tqdm(runNrs.keys())]
tot1_tot2 = pd.DataFrame(
    tot1_tot2,
    columns=[
        "recovery",
        "ToT1",
        "ToT2",
        "absolute loss 2µs",
        "absolute loss 3µs",
        "absolute loss 4µs",
        "relativ loss 2µs",
        "relativ loss 3µs",
        "relativ loss 4µs",
        "relativ loss 5µs",
        "relativ loss 6µs",
        "relativ loss 7µs",
        "relativ loss 8µs",
        "relativ loss 9µs",
        "relativ loss 10µs",
        "scan",
    ],
)
tot1_tot2["ToT1 / ToT2"] = tot1_tot2["ToT1"] / tot1_tot2["ToT2"]
tot1_tot2["ToT2 / ToT1"] = tot1_tot2["ToT2"] / tot1_tot2["ToT1"]
tot1_tot2

# Plots
## Single trace

- yellow mostly straigt line: ToT1
- blue dots: values for ToT2
- green curve of the dots
- vertical blue line: average ToT1 of first 6 values + 475 ns 
- (TODO: is the 475 the time of the super pixel or from the discriminator)

In [ ]:
a = 7
def load_tot_curves(tot1, tot2, **kwargs):
    global df_scans, a
    # define our x-axis from the first scans
    x_axe = np.float_(runNrs["scan0100"]["delays"]) - 1
    model = lambda tau, A, B, c: A * (1 - B * np.exp(-tau / c))

    # get scan_nr for ToT combination
    scan_nr = [k for k, v in runNrs.items() if v['tot1'] == tot1 and v['tot2'] == tot2]

    # for few ToT combinations, there are multiple scans; just take the first one
    scan_nr = scan_nr[0] if scan_nr else 'scan0278'
    df = df_scans.query(f'`scan name` == "{scan_nr}"')

    # omit nans
    y = df[~df['ToT2 (ns)'].isna()]['ToT2 (ns)']
    if len(y) > -offset:
        y = y[offset:]
    x = x_axe[-len(y):]
    p0 = [df['ToT2 (ns)'][-6:].mean(), 1, 1]
    try:
        popt, _ = curve_fit(model, x, y, p0=p0)
    except:
        popt = p0

    return hv.Overlay(
            hv.Curve((x_axe, df['ToT2 (ns)']), label='ToT of 2nd LED')
            * hv.Scatter((x_axe, df['ToT2 (ns)']), label='ToT of 2nd LED').opts(size=5)
            * hv.Curve(
                (x, model(x, *popt)),
                label=f"A(1-B exp(-t/c)):\nA={popt[0]:.1f}, B={popt[1]:.2f}, c={popt[2]:.2f}",).opts(color='green')
            * hv.ErrorBars(np.column_stack([x_axe, df['ToT2 (ns)'], df['ToT2 error']]))
            * hv.Curve((x_axe, df['ToT1 (ns)']), label=f"ToT of 1st LED") 
            #* hv.VLine((df['ToT1 (ns)'][:6].mean()+475)*1e-3).opts(color='blue')  # first 6 TOT1s (blue)
            * hv.VLine((df['ToT1 (ns)'][-10:].mean()+475)*1e-3).opts(color='red')  # last 10 TOT1s (red)
            * hv.HLine(df['ToT1 (ns)'].iloc[0] - df['ToT1 (ns)'][-10:].mean())#.opts(line_width=1)
            ).opts(legend_position="bottom", framewise=True).opts(
                opts.Curve(
                xlabel="delay (µs)",
                ylabel="mean ToT (ns)",
                title=f'{scan_nr}, IKrum={runNrs[scan_nr]["IKrum"]}',
                xlim=(0.5, None),
                logx=True,
                framewise=True
        ))

dmap = hv.DynamicMap(load_tot_curves, kdims=['ToT1','ToT2'])
dmap = dmap.redim.values(ToT1=tot1s, ToT2=tot2s)

dmap.opts(framewise=True)

In [ ]:
# quantify early ToT2

x_axe = np.float_(runNrs["scan0100"]["delays"]) - 1
df_Δt = pd.DataFrame(columns=['ToT1 (ns)', 'ToT2 (ns)', 'Δt (ns)'])
for scan_nr in tqdm(runNrs.keys()):
    # get data 
    df = df_scans.query(f'`scan name` == "{scan_nr}"')
    
    df.reset_index(inplace=True, drop=True)
    tot2_real = x_axe[df['ToT2 (ns)'].first_valid_index()] * 1e3

    dead_time = np.median(df['ToT1 (ns)']) + 475
    ToT1 = df['ToT1 (ns)'][-10:].mean()
    ToT2 = df['ToT2 (ns)'][-10:].mean()
    Δt = tot2_real - dead_time
    Δt = 0. if ToT1 < 1000-475 else Δt
    df_Δt = pd.concat((df_Δt, pd.DataFrame([[ToT1, ToT2, Δt]], columns=df_Δt.columns)))


In [ ]:
xy_hist, x_bins, y_bins, binnumber = binned_statistic_2d(df_Δt['ToT1 (ns)'], df_Δt['ToT2 (ns)'], 
    df_Δt['Δt (ns)'], bins=(range(50, 2150, 100), range(50, 2150, 100)))
a = hv.Image(xy_hist.T[::-1], bounds=[x_bins[0], y_bins[0], x_bins[-1], y_bins[-1]])
a.opts(cmap='coolwarm', logz=False, colorbar=True, title='t(ToT2) (ns) - deadtime')

x_hist, x_bins = np.histogram(df_Δt['Δt (ns)'], bins=15)
x = get_x_axis_from_bins(x_bins) 
b = hv.Curve((x, x_hist))
c = hv.Scatter((x, x_hist))
b.opts(width=400, xlabel='t(ToT2) - Δt (ns)', ylabel='counts')

(a + b * c).opts(shared_axes=False)

## Recovery time

In [ ]:
xy_hist, x_bins, y_bins, binnumber = binned_statistic_2d(tot1_tot2['ToT1'], tot1_tot2['ToT2'], 
    tot1_tot2.recovery, bins=(range(50, 2150, 100), range(50, 2150, 100)))
a = hv.Image(xy_hist.T[::-1], bounds=[x_bins[0], y_bins[0], x_bins[-1], y_bins[-1]])
a.opts(cmap='coolwarm', logz=True, colorbar=True)

x_hist, x_bins = np.histogram(tot1_tot2.recovery, bins=20)
x = get_x_axis_from_bins(x_bins) 
b = hv.Curve((x, x_hist))
c = hv.Scatter((x, x_hist))
b.opts(width=400, xlabel='Recovery time (µs)', ylabel='counts')

(a + b * c).opts(shared_axes=False)

## absolute loss

In [ ]:
x_bins, y_bins = np.arange(50, 2150, 100), np.arange(50, 2150, 100)
x = get_x_axis_from_bins(x_bins)
y = get_x_axis_from_bins(y_bins)
data = []
dims = []
gen = (col for col in tot1_tot2.columns if col.startswith("absolute"))
for col in gen:
    data.append(
        binned_statistic_2d(
            tot1_tot2["ToT1"],
            tot1_tot2["ToT2"],
            tot1_tot2[col],
            bins=(x_bins, y_bins),
        )[0].T
    )
    dims.append(col)
data = np.array(data)
da = xr.DataArray(data, coords=[("Δt", dims), ("ToT1", x), ("ToT2", y)])

In [ ]:
da.hvplot(groupby='Δt', cmap='viridis')

## relativ loss

In [ ]:
x_bins, y_bins = np.arange(50, 2150, 100), np.arange(50, 2150, 100)
x = get_x_axis_from_bins(x_bins)
y = get_x_axis_from_bins(y_bins)
data = []
dims = []
gen = (col for col in tot1_tot2.columns if col.startswith("relativ"))
for col in gen:
    data.append(
        binned_statistic_2d(
            tot1_tot2["ToT1"],
            tot1_tot2["ToT2"],
            tot1_tot2[col],
            bins=(x_bins, y_bins),
        )[0].T
    )
    dims.append(col)
data = np.array(data)
da = xr.DataArray(data, coords=[("Δt", dims), ("ToT1", x), ("ToT2", y)])

In [ ]:
da.hvplot(groupby='Δt', cmap='viridis')

# model

## step 0

In [ ]:
xy_hist, x_bins, y_bins, binnumber = binned_statistic_2d(tot1_tot2['ToT1'], tot1_tot2['ToT2'], 
    #tot1_tot2['relativ loss 2µs'], bins=(range(50, 2150, 100), range(50, 2150, 100)))
    tot1_tot2['relativ loss 3µs'], bins=(range(50, 2150, 100), range(50, 2150, 100)))
x = get_x_axis_from_bins(x_bins)
y = get_x_axis_from_bins(y_bins)

In [ ]:
hv.Overlay(
    hv.Curve((x, xy_hist[:, 14]), label=f"ToT2={y_bins[14]+50}") *
    hv.Curve((x, xy_hist[:, 9]), label=f"ToT2={y_bins[9]+50}") *
	hv.Curve((x, xy_hist[:, 4]), label=f"ToT2={y_bins[4]+50}") *
	hv.Curve((x, xy_hist[:, 1]), label=f"ToT2={y_bins[1]+50}") *
	hv.Curve((x, xy_hist[:, 0]), label=f"ToT2={y_bins[0]+50}") 
).opts(legend_position='top_left').opts(
    opts.Curve(xlabel='ToT1 (ns)', ylabel='relativ loss')
)
#plt.xlabel('ToT1 (ns)')$ *
#plt.ylabel('relativ loss')

## step 1
take the TOT2(TOT1) for every Δt the relative loss is calculated for and fit the slope from a linear function

In [ ]:
plots = []
params = []
for Δt in tqdm(range(2, 11)):
    xy_hist, x_bins, y_bins, binnumber = binned_statistic_2d(tot1_tot2['ToT1'], tot1_tot2['ToT2'], 
                                                             tot1_tot2[f'relativ loss {Δt}µs'], bins=(range(50, 2150, 100), range(50, 2150, 100)))
    ToT1 = x = get_x_axis_from_bins(x_bins)
    y = get_x_axis_from_bins(y_bins)

    for loss, ToT2 in zip(xy_hist.T, x):
        model1 = lambda ToT1, m: m * ToT1
        
        mask = ~np.isnan(loss)
        if mask.any() == True:
            popt1, _ = curve_fit(model1, ToT1[mask], loss[mask])
            params.append([Δt, ToT2, *popt1])

            plots.append(hv.Curve((ToT1, loss), label=f'Δt = {Δt} µs, ToT2 = {ToT2:n}').opts(xlabel='ToT1 (ns)', ylabel="loss (%)", width=400) * 
                         hv.Curve((ToT1, model1(ToT1, *popt1)), label=', '.join([f'{i:.2e}' for i in popt1])) #*
                         #hv.Curve((ToT2, model2(ToT2, *popt2)), label=', '.join([f'{i:.2f}' for i in popt2]))
                        )
params_ToT2 = pd.DataFrame(params, columns=['Δt', 'ToT2', 'm',])


# step 2
fit an exponetial decay for every TOT2 for the Δt

In [ ]:
model = lambda x, A, c: A * np.exp(- x / c)

plots = []
params_expfit = []
for key, df in params_ToT2.groupby("ToT2")['Δt', 'm']:
    popt, _ = curve_fit(model, df['Δt'], df['m'])
    params_expfit.append([key, *popt])
    
    plots.append(hv.Curve(  (df['Δt'], df['m']), label=f'ToT2 = {key}').opts(xlabel='Δt (ns)', ylabel='slope') *
                 hv.Scatter((df['Δt'], df['m']), label=f'ToT2 = {key}').opts(size=5)
                )

params_expfit = pd.DataFrame(params_expfit, columns=['ToT2', 'A', 'c'])
hv.Overlay(plots[::2]).opts(legend_position='left', fontscale=1.5)

## step 3
fit the model $\frac{d}{ToT + B}+g$ from the previously extracted fit parameter $A$

In [ ]:
model = lambda ToT, d, B, g: d / (ToT + B) + g
popt, _ = curve_fit(model, params_expfit['ToT2'][1:], params_expfit['A'][1:], p0=[6, 39, 8e-3])
x_axe = range(200, 2000)

a = hv.Curve((params_expfit['ToT2'], params_expfit['c']), label=f'fit parameter c: median={params_expfit["c"].median():.3f}')
b = (hv.Curve((params_expfit['ToT2'], params_expfit['A']), label=f'fit parameter A').opts(xlabel='ToT2 (ns)', fontscale=1.5) *
     hv.Scatter((params_expfit['ToT2'], params_expfit['A']), label=f'fit parameter A').opts(size=5) *
     hv.Curve((x_axe, model(x_axe, *popt)), label=', '.join([f'{i:.2e}' for i in popt])))

#hv.Layout(a + b).opts(shared_axes=False)
b

## step 4
all parameters are together for the model
$$
ToT_1 \exp(-t / c) \left(\frac{d}{ToT_2 + B} + g\right)
$$

In [ ]:
print(f'c = {params_expfit["c"][2:].median():2f}')
print(f'd = {popt[0]:.2f}')
print(f'B = {popt[1]:.1f}')
print(f'g = {popt[2]:.3f}')

In [ ]:
d = popt[0]
B = popt[1]
g = popt[2]
c = params_expfit["c"].median()
t = 2
model = lambda ToT1, ToT2, t, d, g, B: ToT1 * np.exp(-t / c) * (d / (ToT2 + B) + g)


ToT1 = list(range(100, 2050, 100))
ToT2 = list(range(200, 2050, 100))
H_model = np.array([model(i, ToT2, t, d, g, B) for i in ToT1]).T[::-1]
#a = hv.Image(H_model, bounds=[ToT1[0]-50, ToT2[0]-50, ToT1[-1]+50, ToT2[-1]+50], label=f'model: {t} µs').opts(xlabel="ToT1 (ns)", ylabel="ToT2 (ns)")

xy_exp, x_bins, y_bins, binnumber = binned_statistic_2d(tot1_tot2['ToT1'], tot1_tot2['ToT2'], 
    tot1_tot2[f'relativ loss {t}µs'], bins=(range(50, 2150, 100), range(150, 2150, 100)))
H_exp = xy_exp.T[::-1]

H_diff = H_exp - H_model
#c = hv.Image(H_diff,
#    bounds=(x_bins[0], y_bins[0], x_bins[-1], y_bins[-1])).opts(title=f"differenence (%)at {t}µs", cmap='coolwarm', shared_axes=False)

print(f"min. difference: {np.nanmin(H_diff):.2f}, max. difference: {np.nanmax(H_diff):.2f}")
#(a + b + c).opts(opts.Image(xlabel='ToT1 (ns)', ylabel='ToT2 (ns)', cmap='jet', colorbar=True, clim=(0, 65), fontscale=1.5))

In [ ]:
ToT2[0]

In [ ]:
axis_lim = [ToT1[0]-50, ToT2[0]-50, ToT1[-1]+50, ToT1[-1]+50]
hv.Layout(hv.Image(H_model, bounds=axis_lim).opts(title='model', cmap="viridis") + 
          hv.Image(H_exp, bounds=axis_lim).opts(title='experiment', cmap="viridis") + 
          hv.Image(H_diff, bounds=axis_lim).opts(title='Δ(model, experiment)', cmap="coolwarm")
          ).opts(
              opts.Image(colorbar=True, axiswise=True)
          )